In [1]:
import time
import numpy as np
import gym
from gym import spaces
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import drive, widgets
from IPython.display import display, clear_output

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
IMAGES_DIR = "/content/drive/MyDrive/Assignment1/images"

In [ ]:
MAX_TIMESTEP = 10
MAZE_WIDTH = 4
MAZE_LENGTH = 4
NUM_ACTIONS = 4
NUM_STATES = MAZE_WIDTH * MAZE_LENGTH

In [ ]:
START_POS = np.asarray([3, 0])
FINISH_POS = np.asarray([0, 3])
GOLD_POS = np.asarray([0, 0])
SILVER_POS = np.asarray([3, 3])
OBSTACLE_POS = np.asarray([[2, 0], [1, 2], [1, 3], [3, 2]])

In [ ]:
MAZE = np.zeros((MAZE_WIDTH, MAZE_LENGTH))
MAZE[tuple(START_POS)] = 2
MAZE[tuple(FINISH_POS)] = 3
MAZE[tuple(SILVER_POS)] = 4
MAZE[tuple(GOLD_POS)] = 5
for i in range(len(OBSTACLE_POS)):
    MAZE[tuple(OBSTACLE_POS[i])] = -1